# copied from a previous rawnav repo - needs to be updated for the new workflow

In [1]:
import os, sys, glob, pandas as pd, geopandas as gpd
import pyarrow as pa, pyarrow.parquet as pq
import folium
from folium.plugins import MarkerCluster
from folium import CircleMarker
from IPython.display import IFrame

#### ben imports
import json
import numpy as np
from scipy.spatial import cKDTree
from shapely.geometry import LineString, Point


pd.set_option('display.max_columns', None)
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
%load_ext autoreload
%autoreload 2

#NAD83 Maryland (ft)
wmata_crs = 2248

In [87]:
from ipywidgets import interact
from shapely.ops import cascaded_union, linemerge

In [7]:
#os.path.pardir is equivalent to ".."
#need to append folder 2 levels up in order to load wmatarawnav
sys.path.append(os.path.join( os.path.pardir,os.path.pardir))
import wmatarawnav as wr

In [8]:
# 1.2 Set Global Parameters
############################################
if os.getlogin() == "WylieTimmerman":
    path_working = r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL")
    path_sp = r"C:\Users\WylieTimmerman\Documents\projects_local\wmata_avl_local"
    path_source_data = os.path.join(path_sp,"data","00-raw")
    path_processed_data = os.path.join(path_sp, "data","02-processed")
    path_segments = os.path.join(path_working,"data","02-processed")
elif os.getlogin() == "abibeka":
    path_working = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\Github\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(path_working)
    path_source_data = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\WMATA-AVL\Data"
    path_processed_data = os.path.join(path_source_data, "ProcessedData")
    path_segments = path_processed_data
elif os.getlogin() == "E048374":
    # Working Paths
    path_working = r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL")
    path_source_data = r"\\l-600730\RawNavArchive"
    path_sp = r"C:\Users\E048374\Documents\RawNav"
    path_processed_data = os.path.join(path_sp, "data", "02-processed")
    path_segments = path_processed_data
    
else:
    raise FileNotFoundError("Define the path_working, path_source_data, gtfs_dir, \
                            ZippedFilesloc, and path_processed_data in a new elif block")

## load segments for visualization

In [9]:
bus_lane_edges_loc = "C:\\Users\\E048374\\OneDrive - WMATA\\rawnav_rachel_fork\\WMATA_AVL\\data\\02-processed\\route_52_bus_link_edges.geojson"
bus_lane_edges = gpd.read_file(bus_lane_edges_loc)

In [10]:
#filter for pattern_id = 5201 - south, 5202 - north
bus_lane_edges = (bus_lane_edges[(bus_lane_edges.loc[:,'PATTERN_ID']=='5201')
                    |(bus_lane_edges.loc[:,'PATTERN_ID']=='5202')]).copy()
bus_lane_edges.loc[:,'PATTERN_ID'].unique()

array(['5201', '5202'], dtype=object)

In [11]:
bus_lane_edges.dtypes

PATTERN_ID         object
from_geoid         object
from_stop_seq       int64
to_stop_seq         int64
to_geoid           object
from_node          object
to_node            object
link_code          object
edge_code          object
length            float64
geometry         geometry
dtype: object

In [12]:
#need to change datatype for merge with stop sequence below
bus_lane_edges.loc[:,'from_geoid'] = bus_lane_edges.loc[:,'from_geoid'].astype('int64')
bus_lane_edges.loc[:,'to_geoid'] = bus_lane_edges.loc[:,'to_geoid'].astype('int64')

In [13]:
bus_lane_edges.head(3)

,PATTERN_ID,from_geoid,from_stop_seq,to_stop_seq,to_geoid,from_node,to_node,link_code,edge_code,length,geometry
0,5201,7352,18,19,7243,49745296,5445907574,7352-7243,49745296-5445907574,56.492,"LINESTRING (-77.03222 38.92454, -77.03221 38.9..."
1,5201,7352,18,19,7243,49745297,49745296,7352-7243,49745297-49745296,23.983,"LINESTRING (-77.03224 38.92476, -77.03222 38.9..."
2,5201,7352,18,19,7243,49745301,4887731136,7352-7243,49745301-4887731136,22.908,"LINESTRING (-77.03232 38.92550, -77.03231 38.9..."


## load stop sequence

In [14]:
stop_seq_loc = "C:\\Users\\E048374\\OneDrive - WMATA\\rawnav_rachel_fork\\WMATA_AVL\\data\\02-processed\\route_52_stop_sequence.geojson"
stop_seq = gpd.read_file(stop_seq_loc)

In [15]:
stop_seq.dtypes

ROUTE_ID            object
ROUTE_VERSION        int64
DIRECTION           object
STOPID               int64
GEOID                int64
STOPORDER            int64
STOPSORTORDER        int64
LONGITUDE          float64
LATITUDE           float64
PATTERN_ID          object
GEODESCRIPTION      object
STOP_SEQUENCE      float64
REVENUE_FLAG         int64
geometry          geometry
dtype: object

In [16]:
stop_seq.head(3)

,ROUTE_ID,ROUTE_VERSION,DIRECTION,STOPID,GEOID,STOPORDER,STOPSORTORDER,LONGITUDE,LATITUDE,PATTERN_ID,GEODESCRIPTION,STOP_SEQUENCE,REVENUE_FLAG,geometry
0,52,1,SOUTH,6192,8676,2,1,-77.033157,38.955723,5201,COLORADO AVE NW + 14TH ST NW,1.0,1,POINT (-77.03316 38.95572)
1,52,1,SOUTH,25231,8667,3,2,-77.033531,38.955456,5201,14TH ST + JEFFERSON DR,2.0,1,POINT (-77.03353 38.95546)
2,52,1,SOUTH,21857,8616,4,3,-77.033508,38.954307,5201,14TH ST + INGRAHAM ST,3.0,1,POINT (-77.03351 38.95431)


## join edges to stop sequence 
We're trying to find the stops along each pattern in the bus lane area

In [17]:
from_stop_seq_merge = stop_seq.merge(bus_lane_edges[['PATTERN_ID', 'from_geoid']], how='inner'
                                     , left_on=['PATTERN_ID', 'GEOID']
                                     , right_on=['PATTERN_ID', 'from_geoid'])

to_stop_seq_merge = stop_seq.merge(bus_lane_edges[['PATTERN_ID', 'to_geoid']], how='inner'
                                     , left_on=['PATTERN_ID', 'GEOID']
                                     , right_on=['PATTERN_ID', 'to_geoid'])

tofrom_stop_seq_merge = from_stop_seq_merge.append(to_stop_seq_merge)

In [18]:
reduced_stop_seq = tofrom_stop_seq_merge[['ROUTE_ID', 'ROUTE_VERSION', 'DIRECTION', 'STOPID', 'GEOID',
       'STOPORDER', 'STOPSORTORDER', 'LONGITUDE', 'LATITUDE', 'PATTERN_ID',
       'GEODESCRIPTION', 'STOP_SEQUENCE', 'geometry']].drop_duplicates().copy()

In [19]:
def get_centroid_gpd(gpd_df):
    '''
    '''
    minx = gpd_df.bounds.mean()['minx']
    miny = gpd_df.bounds.mean()['miny']
    maxx = gpd_df.bounds.mean()['maxx']
    maxy = gpd_df.bounds.mean()['maxy']
    avg_x = np.array([minx, maxx]).mean()
    avg_y = np.array([miny, maxy]).mean()
    #return Lat, Long
    return[avg_y, avg_x]

### make a line between stop and nearest rawnav point

In [91]:
esri_imagery = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
esri_attribution = \
    "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, " \
    "UPR-EGP, and the GIS User Community"
this_map = folium.Map(location=get_centroid_gpd(reduced_stop_seq),
                       tiles='cartodbdark_matter', zoom_start=16, max_zoom=25, control_scale=True)
folium.TileLayer(name="EsriImagery", tiles=esri_imagery, attr=esri_attribution,
                 zoom_start=16, max_zoom=25, control_scale=True).add_to(this_map)
folium.TileLayer('cartodbpositron', zoom_start=16, max_zoom=20, control_scale=True).add_to(this_map)
folium.TileLayer('openstreetmap', zoom_start=16, max_zoom=20, control_scale=True).add_to(this_map)

folium.Choropleth(
    bus_lane_edges,
    line_weight=3,
    line_color='blue'
).add_to(this_map)

h = folium.FeatureGroup(name='stop locations')

wr.plot_marker_clusters(this_map, reduced_stop_seq, "LATITUDE","LONGITUDE",h, fill_color='#f79e19')

this_map.add_children(h)

In [21]:
reduced_stop_seq.head(3)

,ROUTE_ID,ROUTE_VERSION,DIRECTION,STOPID,GEOID,STOPORDER,STOPSORTORDER,LONGITUDE,LATITUDE,PATTERN_ID,GEODESCRIPTION,STOP_SEQUENCE,geometry
0,52,1,SOUTH,21627,16203,19,16,-77.032645,38.928445,5201,14TH ST + IRVING ST,16.0,POINT (-77.03265 38.92845)
3,52,1,SOUTH,19066,16662,20,17,-77.032542,38.926777,5201,14TH ST + HARVARD ST,17.0,POINT (-77.03254 38.92678)
4,52,1,SOUTH,14083,7352,21,18,-77.032448,38.925789,5201,14TH ST NW + GIRARD ST NW,18.0,POINT (-77.03245 38.92579)


In [22]:
# remove two geoids outside the bus lane boundary - 7216 & 18197
# doing this visually but should be able to do with a script later
filtered_stop_seq = (reduced_stop_seq[(reduced_stop_seq.loc[:,'GEOID']!=7216)&
                    (reduced_stop_seq.loc[:,'GEOID']!=7216)]).copy()

## test 1 geoid


In [35]:
@interact(geoid_of_interest=filtered_stop_seq.loc[:,'GEOID'].unique())
def plot_stop_with_link(geoid_of_interest):
    one_stop_seq = filtered_stop_seq[filtered_stop_seq.loc[:,'GEOID']==geoid_of_interest].copy()
    one_link = bus_lane_edges[bus_lane_edges.loc[:,'to_geoid']==geoid_of_interest].copy()

    esri_imagery = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
    esri_attribution = \
        "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, " \
        "UPR-EGP, and the GIS User Community"
    this_map = folium.Map(location=get_centroid_gpd(one_link),
                           tiles='cartodbdark_matter', zoom_start=16, max_zoom=25, control_scale=True)
    folium.TileLayer(name="EsriImagery", tiles=esri_imagery, attr=esri_attribution,
                     zoom_start=16, max_zoom=25, control_scale=True).add_to(this_map)
    folium.TileLayer('cartodbpositron', zoom_start=16, max_zoom=20, control_scale=True).add_to(this_map)
    folium.TileLayer('openstreetmap', zoom_start=16, max_zoom=20, control_scale=True).add_to(this_map)

    folium.Choropleth(
        one_link,
        line_weight=3,
        line_color='blue'
    ).add_to(this_map)

    h = folium.FeatureGroup(name='stop location')

    wr.plot_marker_clusters(this_map, one_stop_seq, "LATITUDE","LONGITUDE",h, fill_color='#f79e19')
    return this_map.add_children(h)

interactive(children=(Dropdown(description='geoid_of_interest', options=(16203, 16662, 7352, 7243, 15248, 1524…

In [36]:
geoid_of_interest=16662
one_stop_seq = filtered_stop_seq[filtered_stop_seq.loc[:,'GEOID']==geoid_of_interest].copy()
one_link = bus_lane_edges[bus_lane_edges.loc[:,'to_geoid']==geoid_of_interest].copy()

In [63]:
one_stop_seq

,ROUTE_ID,ROUTE_VERSION,DIRECTION,STOPID,GEOID,STOPORDER,STOPSORTORDER,LONGITUDE,LATITUDE,PATTERN_ID,GEODESCRIPTION,STOP_SEQUENCE,geometry
3,52,1,SOUTH,19066,16662,20,17,-77.032542,38.926777,5201,14TH ST + HARVARD ST,17.0,POINT (-77.03254 38.92678)


In [79]:
one_link_w_stopid = (one_link.merge(one_stop_seq[['GEOID','STOPID','ROUTE_ID','ROUTE_VERSION','DIRECTION']]
                                    ,how='left', left_on='to_geoid',right_on='GEOID')
                     .rename(columns={'STOPID':'stop_id','ROUTE_ID':'route', 'ROUTE_VERSION':'pattern',
                                     'DIRECTION':'direction'})
                     .copy())

In [92]:
one_link_w_stopid

,PATTERN_ID,from_geoid,from_stop_seq,to_stop_seq,to_geoid,from_node,to_node,link_code,edge_code,length,geometry,GEOID,stop_id,route,pattern,direction
0,5201,16203,16,17,16662,1333685838,1439584653,16203-16662,1333685838-1439584653,58.762,"LINESTRING (-77.03260 38.92805, -77.03255 38.9...",16662,19066,52,1,SOUTH
1,5201,16203,16,17,16662,1439584653,783623535,16203-16662,1439584653-783623535,105.806,"LINESTRING (-77.03254 38.92752, -77.03253 38.9...",16662,19066,52,1,SOUTH
2,5201,16203,16,17,16662,5698252199,1333685838,16203-16662,5698252199-1333685838,83.916,"LINESTRING (-77.03267 38.92880, -77.03267 38.9...",16662,19066,52,1,SOUTH


In [88]:
consolidated_link_df = one_link_w_stopid.groupby(['PATTERN_ID','route','pattern', 'direction','from_geoid', 'from_stop_seq'
                           , 'to_stop_seq', 'to_geoid','stop_id', 'link_code']).agg({'length':sum}).reset_index().copy()
consolidated_link_geom = linemerge(cascaded_union(one_link_w_stopid.geometry.values))

consolidated_link_gdf = gpd.GeoDataFrame(consolidated_link_df, crs="EPSG:4326", geometry=[consolidated_link_geom])

In [89]:
consolidated_link_gdf.loc[:,'seg_name_id'] = consolidated_link_gdf.loc[:,'link_code'].copy()
segment_export = consolidated_link_gdf[['seg_name_id', 'length', 'geometry']].copy()

In [94]:
segment_export

,seg_name_id,length,geometry
0,16203-16662,248.484,"LINESTRING (-77.03267 38.92880, -77.03267 38.9..."


In [90]:
file_name = "pattern_5201_segment.geojson"
file_loc = f"C:\\Users\\E048374\\OneDrive - WMATA\\rawnav_rachel_fork\\WMATA_AVL\\data\\02-processed\\{file_name}"
segment_export.to_file(file_loc, driver="GeoJSON")

In [83]:
seg_pattern = consolidated_link_gdf[['seg_name_id','PATTERN_ID', 'route', 'pattern','direction', 'from_geoid', 'from_stop_seq',
       'to_stop_seq', 'to_geoid', 'stop_id']].copy()

In [86]:
seg_pattern

,seg_name_id,PATTERN_ID,route,pattern,direction,from_geoid,from_stop_seq,to_stop_seq,to_geoid,stop_id
0,16203-16662,5201,52,1,SOUTH,16203,16,17,16662,19066


In [85]:
file_name = "test_seg_pattern_5201.csv"
file_loc = f"C:\\Users\\E048374\\OneDrive - WMATA\\rawnav_rachel_fork\\WMATA_AVL\\data\\02-processed\\{file_name}"
seg_pattern.to_csv(file_loc, index=False)